In [1]:
#ProgramName: Daily-06-UpdateMasterDerivedChangingState
#Purpose: Weekly Update Master Catalog - Routine Maintenance
#Author:  Greg Turmel, Director, Data Governance 
#Date:    2020.08.30 - 2021.06.30
#Errata:  0.1 Improvements can be made to script using for/looping through the metadata captured

import os, sys, argparse, csv, pyodbc, sql, time, datetime
import sqlalchemy as db
import errno, pathlib2

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

now01 = datetime.datetime.now()

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

In [2]:
# This segment builds the appropriatte file system structure as a variable driven exercise
# Take time to set your 'eeeeeeeeee' number as variable 'pn' below 
# and set the program directory variable called 'programDirectory' before running
# ===================================================================================================================
pn = r'e201873842'            #This represents the windows system employee login folder - IBM team uses a 9 number
# ===================================================================================================================

programDirectory = 'Daily' # Update this variable to wherever you want the program subfolder/files to be located 
un = r'C:\Users'
cn = r'Documents\Py'
an = r'Master\Process'
bn = r'Master\Reports'

sn = r'Master\sql'
tn = r'config'

#Set a parent directory
parentDirectory = "{}\{}\{}".format(un,pn,cn)
print('Parent Directory is: ', parentDirectory)
mode = 0o666

#Set path location for working with local file(s)
path = os.path.join(parentDirectory, programDirectory,)
pathMP = os.path.join(parentDirectory, programDirectory, an)
pathMR = os.path.join(parentDirectory, programDirectory, bn)
pathMS = os.path.join(parentDirectory, programDirectory, sn)
pathMT = os.path.join(parentDirectory, programDirectory, tn)

procpath = pathMP
csvpath = pathMR
sqlpath = pathMS
configpath = pathMT

try:
    if not os.path.exists(path):
        os.makedirs(path, mode)
        print('Program Directory subfolder has been created: ', programDirectory)
    else:
        print('Directory:', programDirectory, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMP):
        os.makedirs(pathMP, mode)
        print('Program Directory subfolder has been created: ', pathMP)
    else:
        print('Directory:', pathMP, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMR):
        os.makedirs(pathMR, mode)
        print('Program Directory subfolder has been created: ', pathMR)
    else:
        print('Directory:', pathMR, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMS):
        os.makedirs(pathMS, mode)
        print('Program Directory subfolder has been created: ', pathMS)
    else:
        print('Directory:', pathMS, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMT):
        os.makedirs(pathMT, mode)
        print('Program Directory subfolder has been created: ', pathMT)
    else:
        print('Central Config Directory:', pathMT, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

Parent Directory is:  C:\Users\e201873842\Documents\Py
Directory: Daily >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Master\Process >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Master\Reports >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Master\sql >>>>> Note ---- this folder already exists <<<<<
Central Config Directory: C:\Users\e201873842\Documents\Py\Daily\config >>>>> Note ---- this folder already exists <<<<<


In [3]:
#Set up table - remove previous records 
#Comment out if you want to track changes in table by the TodaysDate column
#TypeError: 'NoneType' object is not iterable --- Solution is: SET ANSI_WARNINGS OFF

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor.execute(''' 
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF EXISTS(SELECT 1 FROM dbo.MasterPRODChangeTracking)
BEGIN
   DELETE FROM Greg.dbo.MasterPRODChangeTracking WHERE [TodaysDate] > '19000101';
END
'''
,)
conn.commit()

conn.close()

print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking cleaned out - step complete ***')

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking cleaned out - step complete ***


cursor = conn.cursor()
cursor.execute(''' 

'''
,)
conn.commit()

In [4]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountApplicationData]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***')

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***


#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to zeroStatic, Static, Changing, Retired, Deleted status
cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODCatalog] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED THEN
UPDATE SET DerivedStaticChging=S.DerivedStaticChging
WHEN NOT MATCHED BY TARGET THEN 
INSERT (DatabaseName,SchemaName,TableName,LastUpdated)
VALUES (S.DatabaseName,S.SchemaName,S.TableName,S.LastUpdated);
'''
,)
conn.commit()

conn.close()

print(' *** Note:  database table Greg.dbo.Master change status updated for today ***')